In [14]:
from keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
from keras.utils import np_utils
import time

In [15]:
data_path = 'dataset'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

img_size=100
data=[]
target=[]


for category in categories:
    folder_path = os.path.join(data_path,category)
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:',e)
            
            
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target = np.array(target)

new_target = np_utils.to_categorical(target)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [16]:
print(data.shape)
unique, counts = np.unique(target, return_counts=True)

print(np.asarray((unique, counts)).T)
target = new_target

(1376, 100, 100, 1)
[[  0 690]
 [  1 686]]


In [17]:
np.save('data_new',data)
np.save('target_new',target)

In [59]:
model=Sequential()

model.add(Conv2D(50,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(8192,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))


model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 98, 98, 50)        500       
_________________________________________________________________
activation_31 (Activation)   (None, 98, 98, 50)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 96, 96, 200)       90200     
_________________________________________________________________
activation_32 (Activation)   (None, 96, 96, 200)       0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 48, 48, 200)       0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 46, 46, 100)       180100    
_________________________________________________________________
activation_33 (Activation)   (None, 46, 46, 100)     

In [61]:
# # data=np.load('data.npy')
# # target=np.load('target.npy')

# model=Sequential()

# model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# #The first CNN layer followed by Relu and MaxPooling layers

# model.add(Conv2D(100,(3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# #The second convolution layer followed by Relu and MaxPooling layers

# model.add(Flatten())
# model.add(Dropout(0.5))
# #Flatten layer to stack the output convolutions from second convolution layer
# model.add(Dense(50,activation='relu'))
# #Dense layer of 64 neurons
# model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)
# checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,validation_split=0.2)
#                   callbacks=[checkpoint]
                  

Epoch 1/20


KeyboardInterrupt: 

In [13]:
model = load_model('model-016.model')
face_clsfr= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


source = cv2.VideoCapture('cap.mp4')

frame_width = int(source.get(3)) 
frame_height = int(source.get(4)) 
   
size = (frame_width, frame_height) 
result_cap = cv2.VideoWriter('try1.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 

font = cv2.FONT_HERSHEY_PLAIN
starting_time= time.time()
frame_id = 0

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

classes = None
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

while(True):
    _,frame= source.read() 
    frame_id+=1
    img = frame
    
    height,width,channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
    net.setInput(blob)
    outs = net.forward(outputlayers)


    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x=int(center_x - w/2)
                y=int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence)) 
                class_ids.append(class_id) 

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)


    for i in indexes:
        i = i[0]
        box = boxes[i]
        if class_ids[i]==0:
            label = str(classes[class_id]) 
            cv2.rectangle(frame, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 0), 2)
            cv2.putText(frame, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.1,4)  

    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
#         cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
        

    elapsed_time = time.time() - starting_time
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
        
        
#     cv2.imwrite('LIVE.mp4',img)
    result_cap.write(img)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

AttributeError: 'NoneType' object has no attribute 'shape'